# 第三週加分作業
用課堂練習的小語料`dcard-top100`，實作一個bi-gram語言模型，並且用困惑度來評估它的表現 (三個新句子平均)。<br>
可以用檔案上傳，或 colab 連結。

In [1]:
%pip install jieba

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install emoji

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import math
import jieba
from collections import defaultdict
import re
import emoji

In [4]:
# load dcard-top100
!wget https://raw.githubusercontent.com/lope-classroom/HW1_spacy/77f8a80e300297f4be17c708244a17133227c49c/dcard-top100.csv

--2024-03-13 21:49:31--  https://raw.githubusercontent.com/lope-classroom/HW1_spacy/77f8a80e300297f4be17c708244a17133227c49c/dcard-top100.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... 

connected.
HTTP request sent, awaiting response... 200 OK
Length: 186427 (182K) [text/plain]
Saving to: ‘dcard-top100.csv.3’

dcard-top100.csv.3  100%[===================>] 182.06K  1.09MB/s    in 0.2s    

2024-03-13 21:49:32 (1.09 MB/s) - ‘dcard-top100.csv.3’ saved [186427/186427]



In [5]:
# read dcard data
data = pd.read_csv('dcard-top100.csv')
texts = data['content'].tolist()

## preprocessing

In [6]:
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)


In [7]:
# remove english & non-chinese & emoji
exp = get_emoji_regexp()
clean_texts = []
for text in texts:
    new = re.sub(r'![\u4e00-\u9fa5]', '', text)
    new = re.sub(r'[a-zA-Z\d:\/\.\=\-\>\<]', '', new)
    new = exp.sub(repl='', string=new)
    lines = new.split('\n')
    clean_texts += lines

In [8]:
clean_texts[:5]

['部分回應在 \r', '謝謝各位的留言，我都有看完\r', '好的不好的，我都接受謝謝大家\r', '\r', '\r']

## Bigram

In [9]:
def train_bigram_model(corpus):
    bigram_model = defaultdict(lambda: defaultdict(int))
    for sentence in corpus:
        tokens = jieba.lcut(sentence)  # 使用jieba進行分詞
        for i in range(len(tokens) - 1):
            bigram_model[tokens[i]][tokens[i+1]] += 1
    # Normalize counts to get probabilities
    for first_word in bigram_model:
        total_count = sum(bigram_model[first_word].values())
        for second_word in bigram_model[first_word]:
            bigram_model[first_word][second_word] /= total_count
    return bigram_model

def calculate_perplexity(sentences, bigram_model):
    total_log_probability = 0
    total_words = 0
    for sentence in sentences:
        tokens = jieba.lcut(sentence)  # 使用jieba進行分詞
        for i in range(len(tokens) - 1):
            first_word = tokens[i]
            second_word = tokens[i+1]
            # Calculate log probability of the bigram
            if first_word in bigram_model and second_word in bigram_model[first_word]:
                bigram_probability = bigram_model[first_word][second_word]
                total_log_probability += math.log2(bigram_probability)
            else:
                # Handle unseen bigrams with a small probability
                total_log_probability += math.log2(0.0001)
            total_words += 1
    # Calculate perplexity
    perplexity = 2 ** (-total_log_probability / total_words)
    return perplexity

In [10]:
# Train the bigram model
bigram_model = train_bigram_model(clean_texts)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.683 seconds.
Prefix dict has been built successfully.


## Perplexity

In [11]:
# Generate new sentences and calculate perplexity
new_sentences = ['被拉麵耽誤的炒麵店', '面試尷尬到，應該可以鑽到地核了', '科技業都怎麼教菜鳥']
avg_perplexity = calculate_perplexity(new_sentences, bigram_model)

In [12]:
print("Average Perplexity:", avg_perplexity)

Average Perplexity: 7085.858271190905
